In [ ]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "SIRE" for domain "person" and was generated for All of Us Controlled Tier Dataset v6
dataset_24718657_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                ) 
            )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_24718657_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_24718657",
  "person_24718657_*.csv")
message(str_glue('The data will be written to {person_24718657_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_24718657_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_24718657_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_24718657_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- NULL
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_24718657_person_df <- read_bq_export_from_workspace_bucket(person_24718657_path)

dim(dataset_24718657_person_df)

head(dataset_24718657_person_df, 5)

In [ ]:
dat <- dataset_24718657_person_df
dim(dat)
length(unique(dat$person_id))
table(dat$gender_concept_id)
table(dat$gender)
table(dat$race)
dat2 <- dat %>% select(!contains("concept_id"))

In [ ]:
library(data.table)
covars <- fread("AoU_98K_covariates.tsv")

In [ ]:
sire <- dat2 %>% left_join(covars)
sire <- sire %>% mutate(sire = ifelse(race == "Black or African American" & ethnicity == "Hispanic or Latino","his_black",  
                                    ifelse(race == "Black or African American" & ethnicity == "Not Hispanic or Latino", "notHis_black",  
                                    ifelse(race == "None Indicated" & ethnicity == "Hispanic or Latino", "his",  
                                    ifelse(race == "White" & ethnicity == "Hispanic or Latino","his_white",  
                                    ifelse(race == "White" & ethnicity == "Not Hispanic or Latino","notHis_white", NA ) ) ) )),  
                      hispanic = ifelse(ethnicity == "Hispanic or Latino", 1, 0))

write.table(sire, "self_report_demographics.txt", row.names = F, quote = F, sep = ";")